In [ ]:
import os
import pandas as pd

In [ ]:
data_dir = "./data"
example_submission_path = os.path.join(data_dir, "gender_submission.csv")
train_csv_path = os.path.join(data_dir, "train.csv")
test_csv_path = os.path.join(data_dir, "test.csv")

# Data description

The data has been split into two groups:

- training set (train.csv)
- test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

## Data Dictionary

| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |


## Variable Notes

**pclass**: A proxy for socio-economic status (SES)
- 1st = Upper
- 2nd = Middle
- 3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...
- Sibling = brother, sister, stepbrother, stepsister
- Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...
- Parent = mother, father
- Child = daughter, son, stepdaughter, stepson
- Some children travelled only with a nanny, therefore parch=0 for them.

In [ ]:
train_df = pd.read_csv(train_csv_path)

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
test_df = pd.read_csv(test_csv_path)

In [ ]:
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
def make_ticket(prefix, number):
    return {'ticket_prefix': prefix, 'ticket_number': number}

def transform_ticket_number(ticket_no, use_nan_map=False):
    ticket_set = {
        'LINE'
    }
    if ticket_no in ticket_set:
        return pd.np.nan
    return int(ticket_no)

def transform_ticket_prefix(ticket_prefix):
    if ticket_prefix == 'STON/O2.':
        return 'STON/O 2.'
    return ticket_prefix
    
def split_ticket(ticket):
    parts = ticket.split()
    if len(parts) == 0:
        return make_ticket(pd.np.nan, np.pd.nan)
    elif len(parts) == 1:
        return make_ticket(pd.np.nan, transform_ticket_number(parts[0]))
    return make_ticket(
        transform_ticket_prefix(" ".join(parts[0:-1])),
        transform_ticket_number(parts[-1])
    )

def clean_cabin(cabin):
    col_name = 'cabin_char'
    if any([
        cabin is None,
        cabin == pd.np.nan,
        isinstance(cabin, str) and len(cabin) == 0
    ]):
        return {col_name: pd.np.nan}
    if isinstance(cabin, str):
        return {col_name: cabin[0]}
    return {col_name: pd.np.nan}

In [ ]:
def clean_df(original_df):
    ticket_split_df = pd.DataFrame(original_df['Ticket'].apply(split_ticket).tolist())
    cabin_df = pd.DataFrame(original_df['Cabin'].apply(clean_cabin).tolist())
    
    clean_df = prepare_feature_df(
        original_df.join(ticket_split_df).join(cabin_df)
    )
    return clean_df

def prepare_feature_df(clean_df):
    return clean_df.drop(
        columns=[
            'Ticket',
            'Name',
            
            # TODO add these back in later
            
            # Look at `set(prepared_df['ticket_prefix'])` to see ticket prefixes
            # that might be similar
            'ticket_prefix',
        ]
    )

In [ ]:
from sklearn.model_selection import train_test_split

def clean_train_df(df):
    cleaned_df = clean_df(df)
    X, y = split_train_x_y_df(cleaned_df)
    return split_train_validate(X, y)
    
def split_train_x_y_df(cleaned_train_df):
    y_col_name = 'Survived'
    X = cleaned_train_df.drop(columns=[y_col_name])
    y = cleaned_train_df[[y_col_name]]
    return X, y

def split_train_validate(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.33,
    )
    return X_train, X_valid, y_train, y_valid

In [ ]:
X_train, X_valid, y_train, y_valid = clean_train_df(train_df)
pd.concat([X_train, X_valid])

In [ ]:
cleaned_test_df = clean_df(test_df)
cleaned_test_df

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

def build_preprocessor():
    numerical_cols = [
        'PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'
    ]
    
    categorical_cols = [
        'Sex', 'Embarked',
        'cabin_char', # TODO should this be converted to an ordered numerical value?
    ]
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='constant')

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    return preprocessor

In [ ]:
from sklearn.linear_model import LogisticRegression

def define_model():
    return LogisticRegression()

In [ ]:
from sklearn.metrics import mean_absolute_error

def build_pipeline():
    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', build_preprocessor()),
                                  ('model', define_model())
                                 ])
    return my_pipeline

def evaluate_pipeline(pipeline, X_train, y_train, X_valid, y_valid):
    # Preprocessing of training data, fit model 
    pipeline.fit(X_train, y_train)

    # Preprocessing of validation data, get predictions
    preds = pipeline.predict(X_valid)

    # Evaluate the model
    score = mean_absolute_error(y_valid, preds)
    print('MAE:', score)
    
    pred_df = pd.DataFrame(
        preds,
        index=y_valid.index,
        columns=['Survived']
    )
    
    num_equal = len(y_valid[pred_df['Survived'] == y_valid['Survived']])
    pct_equal = float(num_equal) / float(len(y_valid))
    print('Percent correct pred:', pct_equal)
    
    return pred_df

In [ ]:
def main(train_df, test_df):
    X_train, X_valid, y_train, y_valid = clean_train_df(train_df)
    X_test = clean_df(test_df)
    pipeline = build_pipeline()
    validation_preds = evaluate_pipeline(pipeline, X_train, y_train, X_valid, y_valid)
    
    test_pred = pipeline.predict(X_test)
    test_pred_df = pd.DataFrame(
        test_pred,
        index=X_test.index,
        columns=['Survived']
    )
    
    return {
        'validation': (X_valid, y_valid, validation_preds),
        'test': (X_test, test_pred_df)
    }

In [ ]:
results = main(train_df, test_df)

In [ ]:
pd.read_csv(example_submission_path)

In [ ]:
results['test'][0].join(results['test'][1])[['PassengerId', 'Survived']]